# Exploring eBay Car Sales Data

### This project aims to gather scraped eBay used car sales data, clean it, analyze it. The dataset used war originally scraped and uploaded to Kaggle (find it here: https://www.kaggle.com/orgesleka/used-cars-database/data). I will be using a subset of the data - 50,000 data points from the full dataset - that has been intentionally "dirtied" so that my "cleaning" skills can be refined.

In [1]:
#Import libraries below 
import pandas as pd, numpy as np
from IPython.display import display


In [2]:
#Read the dataset into Python
autos = pd.read_csv("autos.csv", encoding = 'Latin-1')

In [3]:
display(autos.info()) #show info about the columns

display(autos.head(3)) #show the first few rows

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 20 columns):
dateCrawled            50000 non-null object
name                   50000 non-null object
seller                 50000 non-null object
offerType              50000 non-null object
price                  50000 non-null object
abtest                 50000 non-null object
vehicleType            44905 non-null object
yearOfRegistration     50000 non-null int64
gearbox                47320 non-null object
powerPS                50000 non-null int64
model                  47242 non-null object
odometer               50000 non-null object
monthOfRegistration    50000 non-null int64
fuelType               45518 non-null object
brand                  50000 non-null object
notRepairedDamage      40171 non-null object
dateCreated            50000 non-null object
nrOfPictures           50000 non-null int64
postalCode             50000 non-null int64
lastSeen               50000 non-null obj

None

,dateCrawled,name,seller,offerType,price,abtest,vehicleType,yearOfRegistration,gearbox,powerPS,model,odometer,monthOfRegistration,fuelType,brand,notRepairedDamage,dateCreated,nrOfPictures,postalCode,lastSeen
0,2016-03-26 17:47:46,Peugeot_807_160_NAVTECH_ON_BOARD,privat,Angebot,"$5,000",control,bus,2004,manuell,158,andere,"150,000km",3,lpg,peugeot,nein,2016-03-26 00:00:00,0,79588,2016-04-06 06:45:54
1,2016-04-04 13:38:56,BMW_740i_4_4_Liter_HAMANN_UMBAU_Mega_Optik,privat,Angebot,"$8,500",control,limousine,1997,automatik,286,7er,"150,000km",6,benzin,bmw,nein,2016-04-04 00:00:00,0,71034,2016-04-06 14:45:08
2,2016-03-26 18:57:24,Volkswagen_Golf_1.6_United,privat,Angebot,"$8,990",test,limousine,2009,manuell,102,golf,"70,000km",7,benzin,volkswagen,nein,2016-03-26 00:00:00,0,35394,2016-04-06 20:15:37


In the above cell we can see some information about the imported dataset as well as the first few rows of data. We can see that there are 20 columns. Of those columns dateCrawled, dateCreated, and lastSeen are objects when they should be dates or integers, inferring that there may be some incongruities with that column; similarly for price, yearOfRegistration and odometer are objects instead of an integer.

Before devling too deep into the incogruities, I also observed that the column names use camelcase instead of Python's preferred snakecase, which means we can't just replace spaces with underscores. In the next step I will convert the column names from camelcase to snakecase and reword some of the column names based on the data dictionary to be more descriptive.

In [4]:
display(autos.columns) #Print original column names

autos_columns = [
    'date_crawled', 
     'name', 
     'seller', 
     'offer_type', 
     'price', 
     'abtest',
     'vehicle_type', 
     'registration_year', 
     'gearbox', 
     'power_PS', 
     'model',
     'odometer', 
     'registration_month', 
     'fuel_type', 
     'brand',
     'unrepaired_damage', 
     'ad_created', 
     'nr_of_pictures', 
     'postal_code',
     'last_seen'
                ]
autos.columns = autos_columns
display(autos.columns) #Print modified column names
#display(autos.head())

Index(['dateCrawled', 'name', 'seller', 'offerType', 'price', 'abtest',
       'vehicleType', 'yearOfRegistration', 'gearbox', 'powerPS', 'model',
       'odometer', 'monthOfRegistration', 'fuelType', 'brand',
       'notRepairedDamage', 'dateCreated', 'nrOfPictures', 'postalCode',
       'lastSeen'],
      dtype='object')

Index(['date_crawled', 'name', 'seller', 'offer_type', 'price', 'abtest',
       'vehicle_type', 'registration_year', 'gearbox', 'power_PS', 'model',
       'odometer', 'registration_month', 'fuel_type', 'brand',
       'unrepaired_damage', 'ad_created', 'nr_of_pictures', 'postal_code',
       'last_seen'],
      dtype='object')

In the last cell I changed some of the column names to make more sense to me as the analyst, I also changed the names from camelcase to snakecase. As we can see at the end of that last step's result the column names were successfully replaced.

In [5]:
#Review the descriptive statistics below
display(autos.describe(include='all')) #look at the descriptive statistics for all columns, using include='all' to get both categorical and numeric columns
print('Registration Month:')
display(autos.registration_month.value_counts())
print('Power PS:')
display(autos.power_PS.value_counts())

,date_crawled,name,seller,offer_type,price,abtest,vehicle_type,registration_year,gearbox,power_PS,model,odometer,registration_month,fuel_type,brand,unrepaired_damage,ad_created,nr_of_pictures,postal_code,last_seen
count,50000,50000,50000,50000,50000,50000,44905,50000.000000,47320,50000.000000,47242,50000,50000.000000,45518,50000,40171,50000,50000.0,50000.000000,50000
unique,48213,38754,2,2,2357,2,8,NaN,2,NaN,245,13,NaN,7,40,2,76,NaN,NaN,39481
top,2016-03-12 16:06:22,Ford_Fiesta,privat,Angebot,$0,test,limousine,NaN,manuell,NaN,golf,"150,000km",NaN,benzin,volkswagen,nein,2016-04-03 00:00:00,NaN,NaN,2016-04-07 06:17:27
freq,3,78,49999,49999,1421,25756,12859,NaN,36993,NaN,4024,32424,NaN,30107,10687,35232,1946,NaN,NaN,8
mean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2005.073280,NaN,116.355920,NaN,NaN,5.723360,NaN,NaN,NaN,NaN,0.0,50813.627300,NaN
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,105.712813,NaN,209.216627,NaN,NaN,3.711984,NaN,NaN,NaN,NaN,0.0,25779.747957,NaN
min,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1000.000000,NaN,0.000000,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,0.0,1067.000000,NaN
25%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1999.000000,NaN,70.000000,NaN,NaN,3.000000,NaN,NaN,NaN,NaN,0.0,30451.000000,NaN
50%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2003.000000,NaN,105.000000,NaN,NaN,6.000000,NaN,NaN,NaN,NaN,0.0,49577.000000,NaN
75%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2008.000000,NaN,150.000000,NaN,NaN,9.000000,NaN,NaN,NaN,NaN,0.0,71540.000000,NaN


Registration Month:


0     5075
3     5071
6     4368
5     4107
4     4102
7     3949
10    3651
12    3447
9     3389
11    3360
1     3282
8     3191
2     3008
Name: registration_month, dtype: int64

Power PS:


0        5500
75       3171
60       2195
150      2046
140      1884
101      1756
90       1746
116      1646
170      1492
105      1410
125       964
136       955
102       868
163       847
54        759
143       733
131       713
122       710
110       694
109       620
50        604
80        560
177       542
58        506
120       501
115       481
69        475
45        397
95        382
68        380
         ... 
999         1
455         1
442         1
1082        1
678         1
454         1
187         1
262         1
441         1
585         1
460         1
5867        1
9011        1
268         1
236         1
1367        1
24          1
12          1
1771        1
1003        1
587         1
696         1
952         1
1016        1
682         1
650         1
490         1
362         1
153         1
16312       1
Name: power_PS, Length: 448, dtype: int64

- We can see above that there are several columns with only a couple of unique values, inluding: seller, offer_type, abtest, gearbox, and unrepaired_damage. There are also some that will likely have little use in alalysis like nr_of_pictures and power_PS. These are candidates to be dropped.

- From the above describe function we can see that there are several columns we should look further into: odometer, nr_of_pictures, price, registration_year, power_PS, and registration_month.

    - There are several columns that are stored as text that need to be cleaned and stored as numeric data: odometer, and price

    - There are several columns that are stored as text that need to be cleaned and stored as dates: registration_year, registration_month

In [6]:
#For price and odometer we will clean and convert these to numeric columns
autos["price"] = (autos["price"]
                  .str.replace('$','')
                  .str.replace(',','')
                 ).astype(int)

autos["odometer"] = (autos["odometer"]
                  .str.replace('km','')
                  .str.replace(',','')
                 ).astype(int)

autos.rename(columns={'odometer' : 'odometer_km'}, inplace=True) #Add km so there is context for the column (since units were dropped to make it numeric)

In [7]:
#Review the odometer column
display(autos.odometer_km.unique().shape) #see how many unique values
display(autos.odometer_km.describe()) #view min/max/median/mean, etc
display(autos.odometer_km.value_counts()) #see more info

(13,)

count     50000.000000
mean     125732.700000
std       40042.211706
min        5000.000000
25%      125000.000000
50%      150000.000000
75%      150000.000000
max      150000.000000
Name: odometer_km, dtype: float64

150000    32424
125000     5170
100000     2169
90000      1757
80000      1436
70000      1230
60000      1164
50000      1027
5000        967
40000       819
30000       789
20000       784
10000       264
Name: odometer_km, dtype: int64

As we can see, everything looks good in the odometer column

In [8]:
#Review the odometer column
display(autos.price.unique().shape) #see how many unique values
display(autos.price.describe()) #view min/max/median/mean, etc
display(autos.price.value_counts()) #see more info
display(autos.nlargest(20, ['price']) ) #see top values

autos = autos[autos["price"].between(1,350000)]

display(autos.describe(include='all')) #look at the descriptive statistics for all columns, using include='all' to get both categorical and numeric columns

(2357,)

count    5.000000e+04
mean     9.840044e+03
std      4.811044e+05
min      0.000000e+00
25%      1.100000e+03
50%      2.950000e+03
75%      7.200000e+03
max      1.000000e+08
Name: price, dtype: float64

0         1421
500        781
1500       734
2500       643
1000       639
1200       639
600        531
800        498
3500       498
2000       460
999        434
750        433
900        420
650        419
850        410
700        395
4500       394
300        384
2200       382
950        379
1100       376
1300       371
3000       365
550        356
1800       355
5500       340
1250       335
350        335
1600       327
1999       322
          ... 
46200        1
29600        1
13480        1
21700        1
7373         1
3279         1
4286         1
188          1
17830        1
9130         1
910          1
238          1
2671         1
69900        1
151990       1
2479         1
4510         1
86500        1
47499        1
16998        1
27299        1
41850        1
4780         1
686          1
6495         1
20790        1
8970         1
846          1
2895         1
33980        1
Name: price, Length: 2357, dtype: int64

,date_crawled,name,seller,offer_type,price,abtest,vehicle_type,registration_year,gearbox,power_PS,model,odometer_km,registration_month,fuel_type,brand,unrepaired_damage,ad_created,nr_of_pictures,postal_code,last_seen
39705,2016-03-22 14:58:27,Tausch_gegen_gleichwertiges,privat,Angebot,99999999,control,limousine,1999,automatik,224,s_klasse,150000,9,benzin,mercedes_benz,NaN,2016-03-22 00:00:00,0,73525,2016-04-06 05:15:30
42221,2016-03-08 20:39:05,Leasinguebernahme,privat,Angebot,27322222,control,limousine,2014,manuell,163,c4,40000,2,diesel,citroen,NaN,2016-03-08 00:00:00,0,76532,2016-03-08 20:39:05
27371,2016-03-09 15:45:47,Fiat_Punto,privat,Angebot,12345678,control,NaN,2017,NaN,95,punto,150000,0,NaN,fiat,NaN,2016-03-09 00:00:00,0,96110,2016-03-09 15:45:47
39377,2016-03-08 23:53:51,Tausche_volvo_v40_gegen_van,privat,Angebot,12345678,control,NaN,2018,manuell,95,v40,150000,6,NaN,volvo,nein,2016-03-08 00:00:00,0,14542,2016-04-06 23:17:31
47598,2016-03-31 18:56:54,Opel_Vectra_B_1_6i_16V_Facelift_Tuning_Showcar...,privat,Angebot,12345678,control,limousine,2001,manuell,101,vectra,150000,3,benzin,opel,nein,2016-03-31 00:00:00,0,4356,2016-03-31 18:56:54
2897,2016-03-12 21:50:57,Escort_MK_1_Hundeknochen_zum_umbauen_auf_RS_2000,privat,Angebot,11111111,test,limousine,1973,manuell,48,escort,50000,3,benzin,ford,nein,2016-03-12 00:00:00,0,94469,2016-03-12 22:45:27
24384,2016-03-21 13:57:51,Schlachte_Golf_3_gt_tdi,privat,Angebot,11111111,test,NaN,1995,NaN,0,NaN,150000,0,NaN,volkswagen,NaN,2016-03-21 00:00:00,0,18519,2016-03-21 14:40:18
11137,2016-03-29 23:52:57,suche_maserati_3200_gt_Zustand_unwichtig_laufe...,privat,Angebot,10000000,control,coupe,1960,manuell,368,NaN,100000,1,benzin,sonstige_autos,nein,2016-03-29 00:00:00,0,73033,2016-04-06 21:18:11
47634,2016-04-04 21:25:21,Ferrari_FXX,privat,Angebot,3890000,test,coupe,2006,NaN,799,NaN,5000,7,NaN,sonstige_autos,nein,2016-04-04 00:00:00,0,60313,2016-04-05 12:07:37
7814,2016-04-04 11:53:31,Ferrari_F40,privat,Angebot,1300000,control,coupe,1992,NaN,0,NaN,50000,12,NaN,sonstige_autos,nein,2016-04-04 00:00:00,0,60598,2016-04-05 11:34:11


,date_crawled,name,seller,offer_type,price,abtest,vehicle_type,registration_year,gearbox,power_PS,model,odometer_km,registration_month,fuel_type,brand,unrepaired_damage,ad_created,nr_of_pictures,postal_code,last_seen
count,48565,48565,48565,48565,48565.000000,48565,43979,48565.000000,46222,48565.000000,46107,48565.000000,48565.000000,44535,48565,39464,48565,48565.0,48565.000000,48565
unique,46882,37470,2,1,NaN,2,8,NaN,2,NaN,245,NaN,NaN,7,40,2,76,NaN,NaN,38474
top,2016-04-02 11:37:04,Ford_Fiesta,privat,Angebot,NaN,test,limousine,NaN,manuell,NaN,golf,NaN,NaN,benzin,volkswagen,nein,2016-04-03 00:00:00,NaN,NaN,2016-04-07 06:17:27
freq,3,76,48564,48565,NaN,25019,12598,NaN,36102,NaN,3900,NaN,NaN,29368,10336,34775,1887,NaN,NaN,8
mean,NaN,NaN,NaN,NaN,5888.935591,NaN,NaN,2004.755421,NaN,117.197158,NaN,125770.101925,5.782251,NaN,NaN,NaN,NaN,0.0,50975.745207,NaN
std,NaN,NaN,NaN,NaN,9059.854754,NaN,NaN,88.643887,NaN,200.649618,NaN,39788.636804,3.685595,NaN,NaN,NaN,NaN,0.0,25746.968398,NaN
min,NaN,NaN,NaN,NaN,1.000000,NaN,NaN,1000.000000,NaN,0.000000,NaN,5000.000000,0.000000,NaN,NaN,NaN,NaN,0.0,1067.000000,NaN
25%,NaN,NaN,NaN,NaN,1200.000000,NaN,NaN,1999.000000,NaN,71.000000,NaN,125000.000000,3.000000,NaN,NaN,NaN,NaN,0.0,30657.000000,NaN
50%,NaN,NaN,NaN,NaN,3000.000000,NaN,NaN,2004.000000,NaN,107.000000,NaN,150000.000000,6.000000,NaN,NaN,NaN,NaN,0.0,49716.000000,NaN
75%,NaN,NaN,NaN,NaN,7490.000000,NaN,NaN,2008.000000,NaN,150.000000,NaN,150000.000000,9.000000,NaN,NaN,NaN,NaN,0.0,71665.000000,NaN


In the last step I noted that there were several cars with a \$0 price tag and several greater than \$999,990. Those that were above \$999,990 appear to be outliers and possibly erroneous. For the purposes of this analysis I then removed those from the dataset. Looking at the data stats again, the data looks more clean and ready to be analyzed!

In [9]:
display(autos[['date_crawled','ad_created','last_seen']][0:5])
#print(autos['date_crawled'].str[:10])

,date_crawled,ad_created,last_seen
0,2016-03-26 17:47:46,2016-03-26 00:00:00,2016-04-06 06:45:54
1,2016-04-04 13:38:56,2016-04-04 00:00:00,2016-04-06 14:45:08
2,2016-03-26 18:57:24,2016-03-26 00:00:00,2016-04-06 20:15:37
3,2016-03-12 16:58:10,2016-03-12 00:00:00,2016-03-15 03:16:28
4,2016-04-01 14:38:50,2016-04-01 00:00:00,2016-04-01 14:38:50


In [10]:
display(autos.date_crawled.str[:7].value_counts(normalize=True, dropna=False).sort_index())
print('We see here that the dates crawled mostly took place in March 2016')

print('\n')

display(autos.ad_created.str[:7].value_counts(normalize=True, dropna=False).sort_index())
print('We see here that the dates these ads were created mostly took place in March 2016')

print('\n')

display(autos.last_seen.str[:7].value_counts(normalize=True, dropna=False).sort_index())
print('We see here that the dates these ads were seen mostly took place in April 2016')

print('\n')

display(autos.registration_year.describe())
print('We see here that there are some years that look unlikely (i.e. year 1000, year 9999). Because a car can\'t be first registered after the listing was seen, any vehicle with a registration year above 2016 is definitely inaccurate. Determining the earliest valid year is more difficult. Realistically, it could be somewhere in the first few decades of the 1900s.')
print('\n')
print('Let\'s count the number of listings with cars that fall outside the 1900 - 2016 interval and see if it\'s safe to remove those rows entirely, or if we need more custom logic.')
display(autos.registration_year.value_counts(normalize=True, dropna=False).sort_index())
print('\n')
print('Anything earlier than 1910 or later than 2016 looks like it is there in error. 2016 because that is when the dates were crawled. I will remove values less than or greater than those years')

autos = autos[autos.registration_year.between(1910,2016)]
display(autos.registration_year.describe())

2016-03    0.838073
2016-04    0.161927
Name: date_crawled, dtype: float64

We see here that the dates crawled mostly took place in March 2016




2015-06    0.000021
2015-08    0.000021
2015-09    0.000021
2015-11    0.000021
2015-12    0.000041
2016-01    0.000247
2016-02    0.001256
2016-03    0.837496
2016-04    0.160877
Name: ad_created, dtype: float64

We see here that the dates these ads were created mostly took place in March 2016




2016-03    0.424091
2016-04    0.575909
Name: last_seen, dtype: float64

...

count    48565.000000
mean      2004.755421
std         88.643887
min       1000.000000
25%       1999.000000
50%       2004.000000
75%       2008.000000
max       9999.000000
Name: registration_year, dtype: float64

We see here that there are some years that look unlikely (i.e. year 1000, year 9999). Because a car can't be first registered after the listing was seen, any vehicle with a registration year above 2016 is definitely inaccurate. Determining the earliest valid year is more difficult. Realistically, it could be somewhere in the first few decades of the 1900s.


Let's count the number of listings with cars that fall outside the 1900 - 2016 interval and see if it's safe to remove those rows entirely, or if we need more custom logic.


1000    0.000021
1001    0.000021
1111    0.000021
1800    0.000041
1910    0.000103
1927    0.000021
1929    0.000021
1931    0.000021
1934    0.000041
1937    0.000082
1938    0.000021
1939    0.000021
1941    0.000041
1943    0.000021
1948    0.000021
1950    0.000062
1951    0.000041
1952    0.000021
1953    0.000021
1954    0.000041
1955    0.000041
1956    0.000082
1957    0.000041
1958    0.000082
1959    0.000124
1960    0.000474
1961    0.000124
1962    0.000082
1963    0.000165
1964    0.000247
          ...   
2000    0.064985
2001    0.054278
2002    0.051189
2003    0.055575
2004    0.055657
2005    0.060455
2006    0.054978
2007    0.046886
2008    0.045609
2009    0.042932
2010    0.032719
2011    0.033419
2012    0.026974
2013    0.016535
2014    0.013652
2015    0.008072
2016    0.025121
2017    0.028663
2018    0.009678
2019    0.000041
2800    0.000021
4100    0.000021
4500    0.000021
4800    0.000021
5000    0.000082
5911    0.000021
6200    0.000021
8888    0.0000



Anything earlier than 1910 or later than 2016 looks like it is there in error. 2016 because that is when the dates were crawled. I will remove values less than or greater than those years


count    46681.000000
mean      2002.910756
std          7.185103
min       1910.000000
25%       1999.000000
50%       2003.000000
75%       2008.000000
max       2016.000000
Name: registration_year, dtype: float64

## Analysis

When working with data on cars, it's natural to explore variations across different car brands. We can use aggregation to understand the brand column.

In [21]:
top_brands = autos.brand.value_counts(normalize=True, dropna=False).loc[lambda x : x>.05]
display(top_brands)

volkswagen       0.211264
bmw              0.110045
opel             0.107581
mercedes_benz    0.096463
audi             0.086566
ford             0.069900
Name: brand, dtype: float64

Above we see that Volkswagen has 21% of the listings, followed by BMW and Opel with 11%, and so on and so forth. I set the cut off at 5% so that we are looking at the most relevant car brand listings.

In [25]:
top_brands_names = top_brands.index.tolist()
top_brands_names

['volkswagen', 'bmw', 'opel', 'mercedes_benz', 'audi', 'ford']

In [33]:
#Calculate top price for top brands

top_price_by_brand = {}

for brand in top_brands_names:
    brand_sorted = autos[autos['brand'] == brand]
    top_price_by_brand[brand] = brand_sorted.sort_values('price', ascending=False).iloc[0].loc['price']
    
top_price_by_brand

{'audi': 175000,
 'bmw': 259000,
 'ford': 130000,
 'mercedes_benz': 180000,
 'opel': 38990,
 'volkswagen': 64500}

In [32]:
#Calculate mean price for top brands

mean_price_by_brand = {}

for brand in top_brands_names:
    brand_sorted = autos[autos['brand'] == brand]
    mean_price_by_brand[brand] = brand_sorted['price'].mean()
    
display(mean_price_by_brand)

#Alternate method to find means using group_by:
#autos.groupby('brand')['price'].aggregate('mean')

{'audi': 9336.687453600594,
 'bmw': 8332.820517811953,
 'ford': 3749.4695065890287,
 'mercedes_benz': 8628.450366422385,
 'opel': 2975.2419354838707,
 'volkswagen': 5402.410261610221}

Above we observed that in the top 6 brands, there's a distinct price gap.
- Audi, BMW and Mercedes Benz are more expensive
- Ford and Opel are less expensive
- Volkswagen is in between

For the top 6 brands, let's use aggregation to understand the average mileage for those cars and if there's any visible link with mean price:

In [35]:
#Calculate mean mileage for top brands

mean_mileage_by_brand = {}

for brand in top_brands_names:
    brand_sorted = autos[autos['brand'] == brand]
    mean_mileage_by_brand[brand] = brand_sorted['odometer_km'].mean()
    
display(mean_mileage_by_brand)

#Alternate method to find means using group_by:
#autos.groupby('brand')['price'].aggregate('mean')

{'audi': 129157.38678544914,
 'bmw': 132572.51313996495,
 'ford': 124266.01287159056,
 'mercedes_benz': 130788.36331334666,
 'opel': 129310.0358422939,
 'volkswagen': 128707.15879132022}

In [45]:
#Combine the mean_mileage and mean_price into one table

bmp_series = pd.Series(mean_price_by_brand)
#display(bmp_series)

df = pd.DataFrame(bmp_series, columns=['mean_price'])
#display(df)

bmm_series = pd.Series(mean_mileage_by_brand)
#display(bmm_series)

df_2 = pd.DataFrame(bmm_series, columns=['mean_mileage'])
#display(df_2)

df['mean_mileage'] = df_2
display(df.sort_values(['mean_price'], ascending = False))

,mean_price,mean_mileage
audi,9336.687454,129157.386785
mercedes_benz,8628.450366,130788.363313
bmw,8332.820518,132572.513140
volkswagen,5402.410262,128707.158791
ford,3749.469507,124266.012872
opel,2975.241935,129310.035842


Reviewing the above table it does not look like there is a direct correlation of mileage with price. 